In [31]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [32]:
data_path = r'C:\Users\sence\OneDrive\Masaüstü\FAKE_DATA\updated_fake_data.csv'
excel_path = r'C:\Users\sence\OneDrive\Masaüstü\FAKE_DATA\updated_fake_data_with_individual_performance.xlsx'
data = pd.read_csv(data_path)

In [33]:
criteria = ['autonomous_delta_km', 'total_autonomous_seconds', 'interventions', 'total_delta_km']
weights = [0.25, 0.6, 0.05, 0.1]

In [34]:
def calculate_score(row):
    score = 0
    for criterion, weight in zip(criteria, weights):
        score += row[criterion] * weight
    return score

In [35]:
for position in ['driver', 'operator', 'passenger', 'instructor']:
    data[f'{position}_score'] = data.apply(calculate_score, axis=1) * (data[position] == data[position])


In [36]:
employees = pd.concat([data['driver'], data['operator'], data['passenger'], data['instructor']]).unique()
employee_total_points = {employee: 0 for employee in employees}
employee_task_count = {employee: 0 for employee in employees}
employee_scores = {employee: [] for employee in employees}

In [37]:
for index, row in data.iterrows():
    for position in ['driver', 'operator', 'passenger', 'instructor']:
        employee = row[position]
        score = row[f'{position}_score']
        if employee in employee_total_points:
            employee_total_points[employee] += score
            employee_task_count[employee] += 1
            employee_scores[employee].append(score)

In [38]:
employee_performance = pd.DataFrame({
    'Employee': list(employee_total_points.keys()),
    'Total Point': list(employee_total_points.values()),
    'Task Count': list(employee_task_count.values()),
    'Average Point': [sum(employee_scores[employee]) / len(employee_scores[employee]) if len(employee_scores[employee]) > 0 else 0
                      for employee in employee_total_points.keys()]
})


In [39]:
employee_performance['Normalized Point'] = employee_performance['Total Point'] / employee_performance['Task Count']


In [40]:
employee_performance = employee_performance.sort_values('Normalized Point', ascending=False).reset_index(drop=True)
employee_count = len(employee_performance)

In [41]:
def assign_group(rank):
    percentile = rank / employee_count
    if percentile < 0.2:
        return 1, 'Excellent'
    elif percentile < 0.4:
        return 2, 'Good'
    elif percentile < 0.6:
        return 3, 'Average'
    elif percentile < 0.8:
        return 4, 'Below Average'
    else:
        return 5, 'Needs Improvement'

In [42]:
employee_performance['Group'], employee_performance['Performance Status'] = zip(*employee_performance.index.to_series().apply(assign_group))

In [43]:
wb = Workbook()

In [44]:
ws = wb.active
ws.title = 'Individual Performances'
ws.append(['Employee', 'Total Point', 'Task Count', 'Average Point', 'Group', 'Performance Status'])
for r in dataframe_to_rows(employee_performance[['Employee', 'Total Point', 'Task Count', 'Average Point', 'Group', 'Performance Status']], index=False, header=False):
    ws.append(r)

In [45]:
wb.save(excel_path)